In [57]:
import pandas as pd
import json
import math
import hashlib, binascii
import numpy as np
import time
from datetime import datetime
from shutil import copyfile
from os import listdir
import re

import plaid
import sys
sys.path.insert(1, '../private')
import keys as keys
import datetime


In [47]:
def getFiles():
    dlFiles = listdir("./download")
    try:
        dlFiles.remove('.DS_Store')
    except:
        pass
    
    result = []
    for file in dlFiles:
        fileObject={"name":None, "data":None}
        newData = pd.read_csv("./download/"+file, header=None, names=newColumns)
        fileObject["name"] = file.split(".")[0]
        fileObject["data"] = newData
        result.append(fileObject)
    return result


def getFile(file):
    if file == "data":
        return pd.read_csv("./data/data.csv", header=None, names=oldColumns,index_col=False)
    elif file == "processed":
        return pd.read_csv("./data/processed.csv",index_col=False)
    elif file == "maps":
        return pd.read_csv("./data/1to1maps.csv", header=None, names=['item', 'subCategory'])
    elif file == "subCategories":
        return pd.read_csv("./data/categories.csv", header=None, names=['item', 'subCategory'])
    elif file == "categories":
        return pd.read_csv("./data/breakdown.csv", header=None, names=['subCategory', 'category'])
    
def writeFile(file, df):
    if file =="maps":
        df.to_csv('./data/1to1maps.csv', index=False, header=False)  
    elif file=="subCategories":
        df.to_csv('./data/categories.csv', index=False, header=False)  
    elif file =="data":
        df.to_csv('./data/data.csv', index=False, header=False)  

        

def saveDf(df, fileName, path, header = False):
    miliTime = int(round(time.time()))
    readableTime = datetime.utcfromtimestamp(miliTime).strftime('%Y-%m-%d')
    df.to_csv(f'./backup/{fileName}-{readableTime}.csv', index=False, header=header)
    df.to_csv(f'./{path}/{fileName}.csv', index=False, header=header)    

In [48]:
# Hash Data

oldColumns=['date','item','debit','credit','subCategory','hash', 'account']
addedColumns = ['date','item','debit','credit','hash','account']
newColumns=['date','item','debit','credit','card']
processedColumns=['item','category','subCategory','date','year','month','debit','credit','balance', 'account']

def hashit(df):
    hashs = []
    for index, row in df.iterrows():
        h = hashlib.new('ripemd160')
        it = str(row['item'])
        it2= ' '.join(re.findall(r"[\w']+", it))
        h.update(it2.encode())
        h.update(str(row['credit']).encode())
        h.update(str(row['debit']).encode())
        h.update(str(row['date']).encode())
        hashed =h.hexdigest()
        hashs.append(hashed)
    return hashs

def testDf(df):
    assert df.dtypes['debit'] == 'float64'
    assert df.dtypes['credit'] == 'float64'

def fixDf(df):
    global oldColumns
    df['subCategory'].fillna("",inplace=True)
    df=df[oldColumns]
    return df

def findNewItems(old, new, fileName):
    new['hash']= hashit(new)
    hashfound = []
    for index, row in new.iterrows():
        hashfound.append(row['hash'] in old['hash'].values)
    new['hashfound']=hashfound
    
#     new.loc[new['hashfound'] == False, 'account'] = fileName;
    new['account'] = new['card']
    new.loc[new['card']==0, 'account']=fileName;
    new.loc[new['card']==None, 'account']=fileName;

    newItems = new[new['hashfound'] == False]
    print(newItems)
    return newItems

def convertToJsonArray(df):
    columns = df.columns
    result = []
    for i, row in df.iterrows():
        dummy = {}
        for column in columns:
            dummy[column]=row[column]
        result.append(dummy)
    return(result)


def writeToJson(df):
    items = convertToJsonArray(df)
    with open('data/data.json', 'w') as jsonFile:
        json.dump(items, jsonFile)

def listNewItems(files):
    global oldColumns
    old = getFile('data')
    old['subCategory'].fillna("",inplace=True)
    old.fillna(value=0,inplace=True)
    testDf(old)

    combinedAll = old[oldColumns]
    combinedNew = pd.DataFrame(columns=oldColumns)

    for new in files:
        newData = new['data']
        newName = new['name']
        newData.fillna(value=0,inplace=True)
        testDf(newData)

        newItems = findNewItems(combinedAll, newData, newName)
        newToSave = newItems[addedColumns]

        print(f"{newName} - {len(newToSave)} new items found")

        combinedAll = pd.concat([combinedAll, newToSave])
        combinedNew = pd.concat([combinedNew, newToSave])
        
    combinedNew = fixDf(combinedNew)
    return combinedNew

# Process Hashed Data
def processData(newItems,doAll = False):
    if doAll:
        data = getFile('data')
    else:
        newItems.reset_index(inplace=True)
        data = newItems.copy()

    maps = getFile('maps')
    subCategories = getFile('subCategories')
    categories = getFile('categories')

    categoryMap ={}
    for i, row in categories.iterrows():
        categoryMap[row['subCategory']] = row['category']

    data.fillna("", inplace=True)

    subCatArray = []
    # first mapping the 1to1 mappings
    for i, row in data.iterrows():
        if (row['subCategory'] != ""):
            subCatArray.append(row['subCategory'])
        else:
            try:
                index = pd.Index(maps['item']).get_loc(row['item'].rstrip())
                subCategory = maps.loc[index]['subCategory']
                subCatArray.append(subCategory)
            except:
                subCatArray.append("")

    # then mapping all the general categories
    data['subCategory'] = subCatArray
    subCatArray = pd.Series(subCatArray) 


    for i, categoryRow in subCategories.iloc[::-1].iterrows():
        indo = ((data['item'].str.contains(categoryRow['item'])) & (data['subCategory']==""))
        subCatArray[indo] = categoryRow['subCategory']

    data['balance']=data['credit']-data['debit']

    # finally taking care of special categories with logic
    specialCategories = subCategories[subCategories['item'].str.contains('{{')]
    for i, categoryRow in specialCategories.iterrows():
        itemValuePair = categoryRow['item'].replace('}}', '').split('{{')
        indo = (data['item'].str.contains(itemValuePair[0].rstrip()) & (data['balance']==(float(itemValuePair[1]))))
        subCatArray[indo] = categoryRow['subCategory']

    data['subCategory'] = subCatArray
    data['category'] = data['subCategory'].map(categoryMap)
    data['year']= pd.to_datetime(data['date']).dt.year
    data['month']= pd.to_datetime(data['date']).dt.month    
    return data

def runProcess(files):
    newItems = listNewItems(files)
    if(newItems['item'].count() > 0):
        processedData = processData(newItems)   
        dataWithoutCategory = (processedData[processedData['subCategory'] == ""])
        if(len(dataWithoutCategory) == 0):
            processedAlready = getFile('processed')
            processedAll = pd.concat([processedData, processedAlready])
            processedToSave = processedAll[processedColumns].sort_values(by='date', ascending=False)
            saveDf(processedToSave, 'processed', 'data', True)

            dataAll = getFile('data')
            combinedData = pd.concat([dataAll, newItems])
            combinedData = combinedData[oldColumns]
            saveDf(combinedData, 'data', 'data', False)

            writeToJson(processedToSave)

            print("SAVED")
        else:
            print('Found Gaps, NOT SAVED')
            print(dataWithoutCategory[['item','date','balance']])
    #       dataWithoutCategory[['item','date','balance']].to_csv('./processed/not_found.csv')
    else:
        print('no new items')
        
def resetToCurrentData():
    processedData = processData(None, True)  
    processedToSave = processedData[processedColumns].sort_values(by='date', ascending=False)
    writeToJson(processedToSave)
    saveDf(processedToSave, 'processed', 'data', True)


In [1]:
# files should have schema [{'name':string, 'data':pd.DataFrame}, ...]
# files = getFiles()
# runProcess(files)


In [52]:
resetToCurrentData()

In [1076]:
# reset data and process from backup
from shutil import copyfile
copyfile("./backup/data.csv", "./processed/data.csv")
copyfile("./backup/processed.csv", "./processed/processed.csv")

'./processed/processed.csv'

In [1020]:
def changeData(hash, subCategory):
    df = getFile("data")
    df['subCategory'].fillna("",inplace=True)
    df.fillna(value=0,inplace=True)
    df.loc[dz["hash"] == hash, "subCategory"] = subCategory
    return df

df = changeData("bec633716240d8202d8c6e815215fd20cd190bbf", 'abbas')
writeFile("data", df)

## PLAID

In [59]:
client = plaid.Client(client_id = keys.PLAID_CLIENT_ID, secret=keys.PLAID_SECRET,
                      public_key=keys.PLAID_PUBLIC_KEY, environment=keys.PLAID_ENV, api_version='2019-05-29')


/Users/kzahir/anaconda3/lib/python3.6/site-packages/plaid/client.py:66: UserWarning: 
                Development is not intended for production usage.
                Swap out url for https://production.plaid.com
                via Client.config before switching to production
            
  ''')


In [104]:

def accountName(name):
    result = None
    if result == None:
        result= accountList[accountList['accountId']==name]['account'].values[0]
    if (result == "Visa Cad"):
        result = "visa"
    elif (result == 'Personal Line Of Credit'):
        result = "line"
    elif (result=="Eadvantage Savings Account"):
        result = "savings"
    elif (result=="Tfsa Tax Advantage Savings Account"):
        result= "TFSA"
    elif (result== "Personal Chequing Account USD"):
        result = "USD"
    elif (result=="CIBC Smart Account"):
        result = "debit"
        
    return result

def getTrans(start_date, end_date):
        
    response = client.Transactions.get(keys.access_token,
                                       start_date=start_date,
                                       end_date=end_date, count=500)
    print("# Transactions found" , response['total_transactions'])
    
    accountList= pd.DataFrame(columns=['account', 'subtype', 'type', 'balanceA', 'balanceC' , 'accountId'])
    for account in response['accounts']:
        name = account['name']
        subtype = account['subtype']
        type = account['type']
        balanceA= account['balances']['available']
        balanceC= account['balances']['current']
        accountId= account['account_id']

        b = pd.Series({'account':name,'subtype':subtype,'type':type, 'balanceA':balanceA, 'balanceC':balanceC, 'accountId':accountId})
        accountList = accountList.append(b, ignore_index=True )
        
        
    transactions = response['transactions']

    # Manipulate the count and offset parameters to paginate
    # transactions and retrieve all available data
    while len(transactions) < response['total_transactions']:
        response = client.Transactions.get(keysaccess_token,
                                           start_date='2018-01-01',
                                           end_date='2019-02-01',
                                           offset=len(transactions)
                                          )
        transactions.extend(response['transactions'])
    tx= pd.DataFrame(columns=['date', 'item', 'debit','credit', 'card'])
    
    for account in transactions:
        name = account['name']
        date = account['date']
        debit = account['amount'] if account['amount'] >= 0 else 0;
        credit = -account['amount'] if account['amount'] < 0 else 0;        
        accountId= accountName(account['account_id'])

        b = pd.Series({'date':date,'item':name,'debit':debit, 'credit':credit, 'card':accountId})
        tx = tx.append(b, ignore_index=True )
    return tx

In [110]:
tx = getTrans('2019-01-01', '2019-02-01')

# Transactions found 102


In [108]:
def getFiles():
    result = []
    tx = getTrans('2019-01-01', '2019-02-01')
    fileObject={"name":None, "data":None}
    fileObject["name"] = 'plaid'
    fileObject["data"] = tx
    result.append(fileObject)
    return result

In [2]:
# tx['item'].str.upper().str.contains('LON')

In [118]:
files=getFiles()
files[0]['data'].head()

# Transactions found 102


,date,item,debit,credit,card
0,2019-02-01,"LONGO'S # 12 MAPLE, ON",105.51,0,Visa
1,2019-02-01,PREAUTHORIZED DEBIT LN # 1705943330 CIBC LOANS...,0.04,0,Debit
2,2019-01-31,"BUY BUY BABY #3703 WOODBRIDGE, ON",47.63,0,Visa
3,2019-01-31,"DOLLARAMA # 245 MAPLE, ON",13.56,0,Visa
4,2019-01-31,"HUSKY SWEET RIVER 6112 CONCORD, ON",37.60,0,Visa


In [121]:
list(tx['card'].unique())

['Visa', 'Debit', 'USD']

In [649]:
# HASHING UTILITIES
# hash the data file
# old = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# old['custom'].fillna("",inplace=True)
# old.fillna(value=0,inplace=True)
# assert old.dtypes['debit'] == 'float64'
# assert old.dtypes['credit'] == 'float64'
# old['hash']= hashit(old)
# old.to_csv(f'./processed/data.csv', index=False, header=False)

# hash the other files
# for file in dlFiles:
#     old = pd.read_csv("./processed/cards/"+file, header=None,names=oldColumns)
#     old['custom'].fillna("",inplace=True)
#     old.fillna(value=0,inplace=True)
#     assert old.dtypes['debit'] == 'float64'
#     assert old.dtypes['credit'] == 'float64'
#     old['hash']= hashit(old)
#     old.to_csv(f'./processed/{file.split(".")[0]}.{file.split(".")[1]}', index=False, header=False)

# # populate account field in data
# oldColumns=['date','item','debit','credit','custom','hash', 'account']

# data = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# for file in dlFiles:
#     new = pd.read_csv("./processed/"+file, header=None,names=oldColumns)
#     for index, row in new.iterrows():
#         if(row['hash'] in data['hash'].values):
#             data.loc[data['hash'] == row['hash'],'account']=file.split(".")[0]
            
# data.to_csv(f'./processed/data.csv', index=False, header=False) 


# # check for duplicate fields

# data2 = pd.read_csv("./processed/data.csv", header=None,names=oldColumns)
# for index, row in data2.iterrows():
#     if(data2[data2['hash']==row['hash']].count()['hash']>2):
#         print(row['hash'], row['item'], row['debit'])

In [ ]:
# # PROCESS BENCH MARKING
# # TESTING TWO FILES
# old = pd.read_csv("./processed/output-old.csv", index_col=False)
# new = pd.read_csv("./processed/processed.csv", index_col=False)
# old.sort_values(by=['date','debit'], inplace=True)
# new.sort_values(by=['date','debit'], inplace=True)
# old.reset_index( drop=True,inplace=True)
# new.reset_index(drop=True,inplace=True)
# ineq = []
# for i, row in old.iterrows():
#     if (row['subCategory'] != new.loc[i]['subCategory']):
#         if(row['item']=="TIGERDIRECT.CA MARKHAM, ON"):
#             pass
#         elif('WINNERSHOMESENSE' in row['item']):
#             pass
#         else:
#             pass
# #             print(f"{row['subCategory']}, {new.loc[i]['subCategory']} - {row['balance']} {row['item']}{new.loc[i]['item']}")
            

# y = new.groupby('subCategory').sum()['balance']
# x = old.groupby('subCategory').sum()['balance']
# for i,value in enumerate(x):
#     if(value != y[i]):
#         print(value, y[i])

# # one to one map benchmarking
# def try1():    
#     def check1to1(x):
#         try:
#             index = pd.Index(maps['item']).get_loc(x.rstrip())   
#             return maps.loc[index]['subCategory']
#         except:
#             return None

#     data['subCategory'] = data['subCategory'].combine_first(data['item'].apply(check1to1))
    
# def try2():
#     data.fillna("", inplace=True)
#     subCatArray = []
#     # first mapping the 1to1 mappings
#     for i, row in data.iterrows():
#         if (row['subCategory'] != ""):
#             subCatArray.append(row['subCategory'])
#         else:
#             try:
#                 index = pd.Index(maps['item']).get_loc(row['item'].rstrip())
#                 subCategory = maps.loc[index]['subCategory']
#                 subCatArray.append(subCategory)
#             except:
#                 subCatArray.append("")
#     data['subCategory'] = subCatArray
# %timeit try1()
# %timeit try2()

In [6]:
from process import processData, getFile
import pandas as pd

In [3]:
# processData(None, True)  

In [15]:
data = getFile('data')
pr = getFile('processed')

In [37]:
data[data['subCategory'].isnull()]

,date,item,debit,credit,subCategory,hash,account
0,2018-01-03,Point of Sale - Interac RETAIL PURCHASE 990001...,89.04,0.00,NaN,bec633716240d8202d8c6e815215fd20cd190bbf,debit
1,2018-01-03,Internet Banking E-TRANSFER 100430378413 Hedie...,100.00,0.00,NaN,ea0375ada5fb7a67669f70e2f3be753632c4c7dd,debit
2,2018-01-02,Point of Sale - Interac RETAIL PURCHASE 000001...,76.54,0.00,NaN,a3f2845f8c22d3877b4bda1f301b29e0cfc25627,debit
3,2018-01-02,Internet Banking INTERNET DEPOSIT 000000253774,0.00,1570.00,NaN,275e073bde67a0da53e44d941766a077cabcee04,debit
4,2018-01-02,Internet Banking INTERNET TRANSFER 000000264971,1364.80,0.00,NaN,070acc2e7ecfdbdc6c144b67bf717a58adaa7068,debit
...,...,...,...,...,...,...,...
6643,2019-01-07,Internet Banking POWERSTREAM 000000270043,256.77,0.00,NaN,60abee0623c80b532c5b81fbde472e2f62b83f1a,line
6644,2019-01-07,Internet Banking FIDO (SOLUTION INC.) 00000026...,67.80,0.00,NaN,6a6a29d93e91a24e420d75750403be6c2642b2fd,line
6645,2019-01-07,Internet Banking FIDO (SOLUTION INC.) 00000026...,28.25,0.00,NaN,de02b30499797d0460b99d4a0a0fb9f2b379916d,line
6646,2019-01-07,Internet Banking ENBRIDGE GAS INC. 000000266212,131.24,0.00,NaN,674bb10ecb332fc5f021ada58190150b84a625ac,line


In [35]:
pr[pr['item'].str.contains('NEW WOK NEW WOK QFF CONCORD')]
pr[pr['subCategory'].isnull()]

,item,category,subCategory,date,year,month,debit,credit,balance,account
9,"NEW WOK NEW WOK QFF CONCORD, ON",NaN,NaN,2019-03-26,2019,3,29.81,0.0,-29.81,visa
19,Playstation,NaN,NaN,2019-03-25,2019,3,13.49,0.0,-13.49,visa
57,"KID LEE TORONTO, ON",NaN,NaN,2019-03-18,2019,3,15.82,0.0,-15.82,visa
98,"DAAL ROTI MISSISSAUGA, ON",NaN,NaN,2019-03-11,2019,3,185.70,0.0,-185.70,visa
